# Script - Requêtes NoSQL
## Ayoub Errahamani - Rachid Sahli
### BUT SD FA EMS 32 - SAE NoSQL

### Import des librairies

In [1]:
import pymongo
import pandas as pd

### Connexion

In [3]:
client = pymongo.MongoClient()
db = client.classicmodel

### Requête 1 - Lister les clients n’ayant jamais effecuté une commande

In [4]:
sans_commande = db.customers.aggregate([
    {
        "$lookup": {
            "from": "order",
            "localField": "customerNumber",
            "foreignField": "customerNumber",
            "as": "orders"
        }
    },
    {
        "$match": {"orders": {"$size": 0}}
    },
    {
        "$project": {"customerNumber": 1, "customerName": 1, "_id": 0}
    }
])

pd.DataFrame(sans_commande)

,customerNumber,customerName
0,125,Havel & Zbyszek Co
1,168,American Souvenirs Inc
2,169,Porto Imports Co.
3,206,"Asian Shopping Network, Co"
4,223,Natürlich Autos
5,237,ANG Resellers
6,247,Messner Shopping Network
7,273,"Franken Gifts, Co"
8,293,BG&E Collectables
9,303,Schuyler Imports


### Requête 2 - Pour chaque employé, le nombre de clients, le nombre de commandes et le montant total de celles-ci

In [5]:
stats_employees = db.employee.aggregate([
    # Jointure avec Offices
    {
        "$lookup": {
            "from": "offices",
            "localField": "officeCode",
            "foreignField": "officeCode",
            "as": "officeDetails"
        }
    },
    { "$unwind": { "path": "$officeDetails", "preserveNullAndEmptyArrays": True } },

    # Jointure avec Customers
    {
        "$lookup": {
            "from": "customers",
            "localField": "employeeNumber",
            "foreignField": "salesRepEmployeeNumber",
            "as": "customers"
        }
    },
    { "$unwind": { "path": "$customers", "preserveNullAndEmptyArrays": True } },

    # Jointure avec Payments
    {
        "$lookup": {
            "from": "payments",
            "localField": "customers.customerNumber",
            "foreignField": "customerNumber",
            "as": "payments"
        }
    },

    # Jointure avec Orders
    {
        "$lookup": {
            "from": "order",
            "localField": "customers.customerNumber",
            "foreignField": "customerNumber",
            "as": "orders"
        }
    },
    { "$unwind": { "path": "$orders", "preserveNullAndEmptyArrays": True } },

    # Calcul des montants
    {
        "$lookup": {
            "from": "orderDetails",
            "localField": "orders.orderNumber",
            "foreignField": "orderNumber",
            "as": "orderDetails"
        }
    },
    { 
        "$addFields": {
            "orderAmount": {
                "$sum": {
                    "$map": {
                        "input": "$orderDetails",
                        "as": "detail",
                        "in": { 
                            "$multiply": ["$$detail.priceEach", "$$detail.quantityOrdered"] 
                        }
                    }
                }
            }
        }
    },

    # Groupement
    {
        "$group": {
            "_id": "$employeeNumber",
            "firstName": { "$first": "$firstName" },
            "lastName": { "$first": "$lastName" },
            "officeCity": { "$first": "$officeDetails.city" },
            "numberOfCustomers": { "$addToSet": "$customers.customerNumber" },
            "totalOrders": { "$addToSet": "$orders.orderNumber" },
            "totalPayments": { "$sum": { "$sum": "$payments.amount" } },
            "totalOrderAmount": { "$sum": "$orderAmount" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "employeeNumber": "$_id",
            "firstName": 1,
            "lastName": 1,
            "numberOfCustomers": { "$size": "$numberOfCustomers" },
            "totalOrders": { "$size": "$totalOrders" },
            "totalPayments": 1,
        }
    },

    { "$sort": { "employeeNumber": 1 } }
])

r2 = pd.DataFrame(stats_employees)

# Renommage des colonnes
r2.rename(columns={
    'numberOfCustomers': 'TotalCustomer',
    'totalOrders': 'TotalOrder',
    'totalPayments': 'TotalOrderAmount'
}, inplace=True)

# Résultat
r2

,firstName,lastName,TotalOrderAmount,employeeNumber,TotalCustomer,TotalOrder
0,Diane,Murphy,0.00,1002,0,0
1,Mary,Patterson,0.00,1056,0,0
2,Jeff,Firrelli,0.00,1076,0,0
3,William,Patterson,0.00,1088,0,0
4,Gerard,Bondur,0.00,1102,0,0
5,Anthony,Bow,0.00,1143,0,0
6,Leslie,Jennings,12674066.13,1165,6,34
7,Leslie,Thompson,943442.48,1166,6,14
8,Julie,Firrelli,1035043.99,1188,6,14
9,Steve,Patterson,1545990.08,1216,6,18


### Requête 3 - Idem pour chaque bureau (nombre de clients, nombre de commandes et montant total), avec en plus le nombre de clients d’un pays différent, s’il y en a

In [6]:
stats_bureau = db.employee.aggregate([
    {
        "$unwind": {
            "path": "$offices",
            "preserveNullAndEmptyArrays": True
        }
    },
    # Jointure avec les clients
    {
        "$lookup": {
            "from": "customers",
            "localField": "employeeNumber",
            "foreignField": "salesRepEmployeeNumber",
            "as": "customers"
        }
    },
    { "$unwind": { "path": "$customers", "preserveNullAndEmptyArrays": True } },

    # Jointure avec les commandes
    {
        "$lookup": {
            "from": "order",
            "localField": "customers.customerNumber",
            "foreignField": "customerNumber",
            "as": "orders"
        }
    },
    { "$unwind": { "path": "$orders", "preserveNullAndEmptyArrays": True } },

    {
        "$unwind": {
            "path": "$orders.orderDetails",
            "preserveNullAndEmptyArrays": True
        }
    },

    # Calcul du montant total
    {
        "$addFields": {
            "lineTotal": {
                "$multiply": [
                    "$orders.orderDetails.priceEach",
                    "$orders.orderDetails.quantityOrdered"
                ]
            }
        }
    },

    # Calcul des clients provenant de pays différents
    {
        "$addFields": {
            "isDifferentCountry": {
                "$cond": {
                    "if": { "$ne": ["$customers.country", "$offices.country"] },
                    "then": "$customers.customerNumber",
                    "else": None
                }
            }
        }
    },

    # Groupement
    {
        "$group": {
            "_id": "$officeCode",
            "city": { "$first": "$offices.city" },
            "officeCountry": { "$first": "$offices.country" },
            "numberOfCustomers": { "$addToSet": "$customers.customerNumber" },
            "numberOfOrders": { "$addToSet": "$orders.orderNumber" },
            "totalOrderAmount": { "$sum": "$lineTotal" },
            "customersFromDifferentCountries": { "$addToSet": "$isDifferentCountry" }
        }
    },

    # Calcul du nombre de clients uniques
    {
        "$addFields": {
            "numberOfCustomers": { "$size": "$numberOfCustomers" },
            "numberOfOrders": { "$size": "$numberOfOrders" },
            "customersFromDifferentCountries": {
                "$size": {
                    "$filter": {
                        "input": "$customersFromDifferentCountries",
                        "as": "customer",
                        "cond": { "$ne": ["$$customer", None] }
                    }
                }
            }
        }
    },

    {
        "$project": {
            "_id": 0,
            "officeCode": "$_id",
            "city": 1,
            "officeCountry": 1,
            "numberOfCustomers": 1,
            "numberOfOrders": 1,
            "totalOrderAmount": 1,
            "customersFromDifferentCountries": 1
        }
    },

    {
        "$sort": { "officeCode": 1 }
    }
])

r3 = pd.DataFrame(stats_bureau)

# Renommage des colonnes
r3.rename(columns={
    'totalOrderAmount': 'TotalamountOrder',
    'numberOfOrders': 'TotalOrder',
    'totalOrderAmount': 'TotalamountOrder',
    'numberOfCustomers':'TotalCustomer'
}, inplace=True)

# Résultat
r3

,city,officeCountry,TotalCustomer,TotalOrder,TotalamountOrder,customersFromDifferentCountries,officeCode
0,San Francisco,USA,12,48,1588293.29,0,1.0
1,Boston,USA,12,32,987774.17,0,2.0
2,NYC,USA,15,39,1275993.95,3,3.0
3,Paris,France,29,106,3404055.56,17,4.0
4,Tokyo,Japan,5,16,503957.58,3,5.0
5,Sydney,Australia,10,38,1281705.83,5,6.0
6,London,UK,17,47,1604168.80,12,7.0


### Requête 4 - Pour chaque produit, donner le nombre de commandes, la quantité totale commandée, et le nombre de clients différents

In [7]:
stats_produits = db.order.aggregate([
    {
        "$unwind": "$orderDetails"
    },
    # Groupement
    {
        "$group": {
            "_id": "$orderDetails.product.productCode",
            "productName": { "$first": "$orderDetails.product.productName" },
            "numberOfOrders": { "$addToSet": "$orderNumber" },
            "totalQuantityOrdered": { "$sum": "$orderDetails.quantityOrdered" },
            "uniqueCustomers": { "$addToSet": "$customerNumber" }
        }
    },

    # Calcul des tailles pour le nombre de commandes et de clients distincts
    {
        "$project": {
            "_id": 0,
            "productCode": "$_id",
            "productName" : 1,
            "numberOfOrders": { "$size": "$numberOfOrders" },
            "totalQuantityOrdered": 1,
            "numberOfDistinctCustomers": { "$size": "$uniqueCustomers" }
        }
    },

    { "$sort": { "productCode": 1 } }
])

r4 = pd.DataFrame(stats_produits)

# Renommage des colonnes
r4.rename(columns={
    'totalQuantityOrdered': 'Totalquantity',
    'numberOfDistinctCustomers': 'TotalCustomer',
    'numberOfOrders': 'TotalOrder',
}, inplace=True)

# Résultat
r4

,productName,Totalquantity,productCode,TotalOrder,TotalCustomer
0,[1969 Harley Davidson Ultimate Chopper],1026,[S10_1678],28,26
1,[1952 Alpine Renault 1300],961,[S10_1949],28,27
2,[1996 Moto Guzzi 1100i],999,[S10_2016],28,26
3,[2003 Harley-Davidson Eagle Drag Bike],985,[S10_4698],28,25
4,[1972 Alfa Romeo GTA],1000,[S10_4757],28,27
...,...,...,...,...,...
104,[The Titanic],952,[S700_3505],27,22
105,[The Queen Mary],883,[S700_3962],27,24
106,[American Airlines: MD-11S],1073,[S700_4002],28,26
107,[Boeing X-32A JSF],960,[S72_1253],28,27


### Requête 5 - Donner le nombre de commande pour chaque pays, ainsi que le montant total des commandes et le montant total payé : on veut conserver les clients n’ayant jamais commandé dans le résultat final

In [9]:
stats_pays = db.customers.aggregate([
    # Jointure avec les commandes
    {
        "$lookup": {
            "from": "order",
            "localField": "customerNumber",
            "foreignField": "customerNumber",
            "as": "orders"
        }
    },
    { "$unwind": { "path": "$orders", "preserveNullAndEmptyArrays": True } },

    {
        "$unwind": {
            "path": "$orders.orderDetails",
            "preserveNullAndEmptyArrays": True
        }
    },

    # Jointure avec Payments
    {
        "$lookup": {
            "from": "payments",
            "localField": "customerNumber",
            "foreignField": "customerNumber",
            "as": "payments"
        }
    },
    { "$unwind": { "path": "$payments", "preserveNullAndEmptyArrays": True } },

    # Calcul
    {
        "$addFields": {
            "lineTotal": {
                "$cond": {
                    "if": { "$and": ["$orders.orderDetails.priceEach", "$orders.orderDetails.quantityOrdered"] },
                    "then": { 
                        "$multiply": [
                            "$orders.orderDetails.priceEach", 
                            "$orders.orderDetails.quantityOrdered"
                        ]
                    },
                    "else": 0
                }
            },
            "paymentAmount": {
                "$ifNull": ["$payments.amount", 0]
            }
        }
    },

    {
        "$group": {
            "_id": "$country",
            "numberOfOrders": { 
                "$addToSet": "$orders.orderNumber" 
            },
            "totalOrderAmount": { 
                "$sum": "$lineTotal" 
            },
            "totalPaidAmount": { 
                "$sum": "$paymentAmount"
            }
        }
    },

    {
        "$addFields": {
            "numberOfOrders": { "$size": "$numberOfOrders" }
        }
    },

    {
        "$project": {
            "_id": 0,
            "country": "$_id",
            "numberOfOrders": 1,
            "totalOrderAmount": 1,
            "totalPaidAmount": 1
        }
    },

    { "$sort": { "country": 1 } }
])

r5 = pd.DataFrame(stats_pays)

# Renommage des colonnes
r5.rename(columns={
    'country': 'Country',
    'numberOfOrders': 'TotalOrder',
    'totalPaidAmount': 'Totalpayment',
}, inplace=True)

# Résultat
r5

,TotalOrder,totalOrderAmount,Totalpayment,Country
0,19,2182269.38,2.482541e+07,Australia
1,7,606187.59,4.090982e+06,Austria
2,7,283705.44,1.931535e+06,Belgium
3,7,448157.12,4.487022e+06,Canada
4,7,781357.50,7.001114e+06,Denmark
5,9,988745.73,1.009620e+07,Finland
6,37,3160296.75,3.141444e+07,France
7,7,576293.44,4.971661e+06,Germany
8,2,48784.36,7.805498e+05,Hong Kong
9,2,115512.86,9.241029e+05,Ireland


### Requête 6 - On veut la table de contigence du nombre de commande entre la ligne de produits et le pays du client

In [226]:
order_contingence = db.order.aggregate([
    { "$unwind": "$orderDetails" },

    # Jointure avec Customers
    {
        "$lookup": {
            "from": "customers",
            "localField": "customerNumber",
            "foreignField": "customerNumber",
            "as": "customer"
        }
    },
    
    { "$unwind": "$customer" },

    # Filtrer les documents invalides
    {
        "$match": {
            "orderDetails.product.productLine": { "$exists": True, "$ne": None },
            "customer.country": { "$exists": True, "$ne": None }
        }
    },

    # Groupement par productLine et country
    {
        "$group": {
            "_id": {
                "productLine": "$orderDetails.product.productLine",
                "country": "$customer.country"
            },
            "uniqueOrders": { "$addToSet": "$orderNumber" }
        }
    },

    # Comptage et projection pour nettoyage
    {
        "$project": {
            "_id": 0,
            "productLine": { "$arrayElemAt": ["$_id.productLine", 0] },
            "country": "$_id.country",
            "NumberOfOrders": { "$size": "$uniqueOrders" }
        }
    },

    { "$sort": { "productLine": 1, "country": 1 } }
])

r6 = pd.DataFrame(order_contingence)

# Renommage des colonnes
r6.rename(columns={
    'NumberOfOrders': 'nb_commandes',
}, inplace=True)

# Résultat
r6

,productLine,country,nb_commandes
0,Classic Cars,Australia,12
1,Classic Cars,Austria,5
2,Classic Cars,Belgium,2
3,Classic Cars,Canada,6
4,Classic Cars,Denmark,5
...,...,...,...
121,Vintage Cars,Singapore,4
122,Vintage Cars,Spain,22
123,Vintage Cars,Sweden,4
124,Vintage Cars,UK,10


### Requête 7 - On veut la même table croisant la ligne de produits et le pays du client, mais avec le montant total payé dans chaque cellule

In [231]:
produits_contingence = db.order.aggregate([

    { "$unwind": "$orderDetails" },

    # Jointure avec Customers
    {
        "$lookup": {
            "from": "customers",
            "localField": "customerNumber",
            "foreignField": "customerNumber",
            "as": "customer"
        }
    },
    { "$unwind": "$customer" },

    # Jointure avec Payments
    {
        "$lookup": {
            "from": "payments",
            "localField": "customer.customerNumber",
            "foreignField": "customerNumber",
            "as": "payments"
        }
    },
    { "$unwind": { "path": "$payments", "preserveNullAndEmptyArrays": True } },

    # Filtrer les documents invalides
    {
        "$match": {
            "orderDetails.product.productLine": { "$exists": True, "$ne": None },
            "customer.country": { "$exists": True, "$ne": None }
        }
    },

    # Regrouper par productLine et country pour calculer la somme des paiements
    {
        "$group": {
            "_id": {
                "productLine": "$orderDetails.product.productLine",
                "country": "$customer.country"
            },
            "totalPaidAmount": { "$sum": "$payments.amount" }
        }
    },

    {
        "$project": {
            "_id": 0,
            "country": "$_id.country",
            "productLine": { "$arrayElemAt": ["$_id.productLine", 0] },
            "totalPaidAmount": 1
        }
    },

    { "$sort": { "productLine": 1, "country": 1 } }
])

r7 = pd.DataFrame(produits_contingence)

# Renommage des colonnes
r7.rename(columns={
    'totalPaidAmount': 'Totalamount',
}, inplace=True)

# Résultat
r7

,Totalamount,country,productLine
0,7504795.97,Australia,Classic Cars
1,1884419.42,Austria,Classic Cars
2,166880.87,Belgium,Classic Cars
3,774924.01,Canada,Classic Cars
4,3678313.22,Denmark,Classic Cars
...,...,...,...
121,1941227.28,Singapore,Vintage Cars
122,39878490.66,Spain,Vintage Cars
123,1377094.16,Sweden,Vintage Cars
124,4673162.78,UK,Vintage Cars


### Requête 8 - Donner les 10 produits pour lesquels la marge moyenne est la plus importante (cf buyPrice et priceEach)

In [236]:
dix_produits = db.order.aggregate([
    
    { "$unwind": "$orderDetails" },

    { "$unwind": "$orderDetails.product" },

    # Calcule de la marge pour chaque vente
    {
        "$addFields": {
            "profitMargin": { 
                "$subtract": [
                    "$orderDetails.priceEach", 
                    "$orderDetails.product.buyPrice"
                ]
            }
        }
    },

    # Grouper par produit pour calculer la marge moyenne
    {
        "$group": {
            "_id": {
                "productCode": "$orderDetails.product.productCode",
                "productName": "$orderDetails.product.productName"
            },
            "averageMargin": { "$avg": "$profitMargin" },
            "totalMargin": { "$sum": "$profitMargin" },
            "totalSales": { "$sum": 1 } 
        }
    },

    # Trier les produits par marge moyenne décroissante
    { "$sort": { "averageMargin": -1 } },

    # 10 produits avec la plus grande marge moyenne
    { "$limit": 10 },
    
    {
        "$project": {
            "ProductCode": "$_id.productCode",
            "ProductName": "$_id.productName",
            "AverageMargin": "$averageMargin",
            "_id": 0
        }
    }
])

# Résultat
pd.DataFrame(dix_produits)

,ProductCode,ProductName,AverageMargin
0,S10_1949,1952 Alpine Renault 1300,99.006429
1,S10_4698,2003 Harley-Davidson Eagle Drag Bike,95.235000
2,S18_3232,1992 Ferrari 360 Spider red,83.334906
3,S12_2823,2002 Suzuki XREO,83.201429
4,S18_2795,1928 Mercedes-Benz SSK,82.696786
5,S12_1108,2001 Ferrari Enzo,81.043704
6,S12_3891,1969 Ford Falcon,77.335926
7,S18_3685,1948 Porsche Type 356 Roadster,72.636800
8,S18_2870,1999 Indy 500 Monte Carlo SS,71.794400
9,S18_1749,1917 Grand Touring Sedan,70.432800


### Requête 9 - Lister les produits (avec le nom et le code du client) qui ont été vendus à perte :
            - Si un produit a été dans cette situation plusieurs fois, il doit apparaître plusieurs fois,
            - Une vente à perte arrive quand le prix de vente est inférieur au prix d’achat

In [238]:
produits_perte = db.order.aggregate([
    
    { "$unwind": "$orderDetails" },
    
    { "$unwind": "$orderDetails.product" },
    
    # Jointure avec Customers
    {
        "$lookup": {
            "from": "customers",
            "localField": "customerNumber",
            "foreignField": "customerNumber",
            "as": "customer"
        }
    },
    
    { "$unwind": "$customer" },
    
    # Filtre des documents où le prix de vente est inférieur au prix d'achat
    {
        "$match": {
            "$expr": { "$lt": ["$orderDetails.priceEach", "$orderDetails.product.buyPrice"] }
        }
    },
    
    {
        "$project": {
            "ProductCode": "$orderDetails.product.productCode",
            "ProductName": "$orderDetails.product.productName",
            "CustomerName": "$customer.customerName",
            "CustomerNumber": "$customer.customerNumber",
            "PriceEach": "$orderDetails.priceEach",
            "BuyPrice": "$orderDetails.product.buyPrice",
            "_id": 0
        }
    }

])

# Résultat
pd.DataFrame(produits_perte)

,ProductCode,ProductName,CustomerName,CustomerNumber,PriceEach,BuyPrice
0,S10_4962,1962 LanciaA Delta 16V,Online Diecast Creations Co.,363,61.99,103.42
1,S18_2957,1934 Ford V8 Coupe,Online Diecast Creations Co.,363,29.87,34.35
2,S18_3136,18th Century Vintage Horse Carriage,Online Diecast Creations Co.,363,47.04,60.74
3,S12_3148,1969 Corvair Monza,Vitachrome Inc.,181,54.33,89.14
4,S18_2319,1964 Mercedec Tour Bus,Vitachrome Inc.,181,37.48,74.86
...,...,...,...,...,...,...
74,S10_4962,1962 LanciaA Delta 16V,"Anna's Decorations, Ltd",276,46.90,103.42
75,S12_1666,1958 Setra Bus,"Anna's Decorations, Ltd",276,63.20,77.90
76,S18_2949,1913 Ford Model T Speedster,"Anna's Decorations, Ltd",276,45.25,60.78
77,S18_2238,1998 Chrysler Plymouth Prowler,"Down Under Souveniers, Inc",323,69.81,101.51


### Requête 10 - (bonus) Lister les clients pour lesquels le montant total payé est supérieur aux montants totals des achats

In [240]:
bonus = db.customers.aggregate([
    
    # Jointure avec la collection Orders
    {
        "$lookup": {
            "from": "order",
            "localField": "customerNumber",
            "foreignField": "customerNumber",
            "as": "orders"
        }
    },

    # Jointure avec la collection Payments
    {
        "$lookup": {
            "from": "payments",
            "localField": "customerNumber",
            "foreignField": "customerNumber",
            "as": "payments"
        }
    },

    # Décomposer les commandes pour accéder aux détails
    { "$unwind": { "path": "$orders", "preserveNullAndEmptyArrays": True } },

    # Décomposer les détails des commandes
    { "$unwind": { "path": "$orders.orderDetails", "preserveNullAndEmptyArrays": True } },

    # Calculer le montant total des achats pour chaque détail de commande
    {
        "$addFields": {
            "orderAmount": {
                "$multiply": [
                    { "$ifNull": ["$orders.orderDetails.quantityOrdered", 0] },
                    { "$ifNull": ["$orders.orderDetails.priceEach", 0] }
                ]
            }
        }
    },

    # Groupement
    {
        "$group": {
            "_id": {
                "customerNumber": "$customerNumber",
                "customerName": "$customerName"
            },
            "totalOrderAmount": { "$sum": "$orderAmount" },
            "totalPaidAmount": {
                "$sum": { "$ifNull": [{ "$arrayElemAt": ["$payments.amount", 0] }, 0] }
            }
        }
    },

    # Filtre des clients où totalPaidAmount > totalOrderAmount
    {
        "$match": {
            "$expr": { "$gt": ["$totalPaidAmount", "$totalOrderAmount"] }
        }
    },

    
    {
        "$project": {
            "CustomerNumber": "$_id.customerNumber",
            "CustomerName": "$_id.customerName",
            "TotalOrderAmount": "$totalOrderAmount",
            "TotalPaidAmount": "$totalPaidAmount",
            "_id": 0
        }
    },

    
    { "$sort": { "TotalPaidAmount": 1 } }
])

pd.DataFrame(bonus)

,CustomerNumber,CustomerName,TotalOrderAmount,TotalPaidAmount
0,219,Boards & Toys Co.,9129.35,11961.60
1,103,Atelier graphique,24179.96,37155.86
2,473,Frau da Collezione,28951.91,58231.84
3,198,Auto-Moto Classics Inc.,26479.26,92228.24
4,216,Enaco Distributors,78411.86,97041.60
...,...,...,...,...
90,187,"AV Stores, Co.",157807.81,2949208.11
91,151,Muscle Machine Inc,197736.94,3087172.32
92,321,Corporate Gift Ideas Co.,149882.50,3922834.08
93,124,Mini Gifts Distributors Ltd.,654858.04,9328744.80
